## Preambule

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from tqdm import tqdm

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Input

In [2]:
rules_b = ['GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC']
rules_a = rules_b

## Read data

In [3]:
xr_dataread = xr.open_dataset("K:/ECEMF/T5.2/xr_dataread.nc")
xr_temps = xr.open_dataset("K:/ECEMF/T5.2/xr_temps.nc")
#xr_2030 = xr.open_dataset("X:/user/dekkerm/Projects/ECEMF_T5.2/Data/xr_alloc_2030.nc")

In [4]:
df_eng = pd.read_csv('X:/user/dekkerm/Data/ENGAGE/PolicyScenarios/engage-internal_snapshot_1695642798.csv')
df_eng = df_eng[df_eng.Variable == 'Emissions|Kyoto Gases']
df_eng = df_eng.reset_index(drop=True)
regions_df = np.array(df_eng.Region)
regions_df[regions_df == "Argentine Republic"] = 'ARG'
regions_df[regions_df == "Canada"] = 'CAN'
regions_df[regions_df == "Commonwealth of Australia"] = 'AUS'
regions_df[regions_df == "Federative Republic of Brazil"] = 'BRA'
regions_df[regions_df == "People's Repulic of China"] = 'CHN'
regions_df[regions_df == "European Union (28 member countries)"] = 'EU'
regions_df[regions_df == "Republic of India"] = 'IND'
regions_df[regions_df == "Republic of Indonesia"] = 'IDN'
regions_df[regions_df == "State of Japan"] = 'JPN'
regions_df[regions_df == "Russian Federation"] = 'RUS'
regions_df[regions_df == "Kingdom of Saudi Arabia"] = 'SAU'
regions_df[regions_df == "Republic of South Africa"] = 'ZAF'
regions_df[regions_df == "Republic of Korea (South Korea)"] = 'KOR'
regions_df[regions_df == "United Mexican States"] = 'MEX'
regions_df[regions_df == "Republic of Turkey"] = 'TUR'
regions_df[regions_df == "United States of America"] = 'USA'
regions_df[regions_df == "Viet Nam "] = 'VNM'
df_eng.Region = regions_df
df = pd.read_excel("X:/user/dekkerm/Data//UNFCCC_Parties_Groups_noeu.xlsx", sheet_name = "Country groups")
countries_iso = np.array(df["Country ISO Code"])
group_eu = countries_iso[np.array(df["EU"]) == 1]

In [5]:
curpol = "GP_CurPol_T45"
ndc = "GP_NDC2030_T45"
nz = "GP_Glasgow"
df_eng_ref = df_eng[['Model', 'Scenario', 'Region']+list(df_eng.keys()[5:])]
df_eng_ref = df_eng_ref[df_eng_ref.Scenario.isin([curpol, ndc, nz])]
scen = np.array(df_eng_ref.Scenario)
scen[scen == ndc] = 'NDC'
scen[scen == curpol] = 'CurPol'
scen[scen == nz] = 'NetZero'
reg = np.array(df_eng_ref.Region)
reg[reg == 'World'] = 'WORLD'
df_eng_ref['Scenario'] = scen
df_eng_ref['Region'] = reg
dummy = df_eng_ref.melt(id_vars=["Scenario", "Model", "Region"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Scenario", "Model", "Region", "Time"])
xr_eng = xr.Dataset.from_dataframe(dummy)
xr_eng = xr_eng.reindex(Time = np.arange(1850, 2101))
xr_eng = xr_eng.interpolate_na(dim="Time", method="linear")

## Read memberstate info

In [6]:
for cty_i, cty in tqdm(enumerate(group_eu)):
    ds = xr.open_dataset("K:/ECEMF/T5.2/Allocations/" + "xr_alloc_"+cty+".nc").sel(Temperature=1.5, Risk=0.5).expand_dims(Region=[cty])
    if cty_i == 0:
        xrt = ds.copy()
    else:
        xrt = xr.merge([xrt, ds])
    ds.close()
xrt = xrt.astype("float32")

27it [00:41,  1.53s/it]


## Plot

In [7]:
rulecolors = ['sienna', 'goldenrod', 'forestgreen', 'mediumvioletred', 'steelblue', 'tomato', 'silver']
rules = ['GF', 'PC', 'PCC', 'ECPC', 'AP', 'GDR']
standard_settings = {"TrajUnc":"Medium",
                     "NegEmis":0.5,
                     "Convergence_year":2050}

In [8]:
data = xrt.sel(**standard_settings, Time=np.arange(2021, 2100), Region=group_eu).mean(dim='Scenario').sum(dim='Time')/1e3

In [10]:
fig = make_subplots(rows=2, cols=3, #subplot_titles=('Grandfathering (GF)',
                                                    # 'Per Capita (PC)',
                                                    # 'Per Capita Convergence (PCC)',
                                                    # 'Equal Cumulative Per Capita',
                                                    # 'Ability to pay (AP)',
                                                    # 'Greenhouse Development Rights'), 
                                    specs=[[{'type': 'choropleth'}, {'type': 'choropleth'}, {'type': 'choropleth'}],
                                           [{'type': 'choropleth'}, {'type': 'choropleth'}, {'type': 'choropleth'}]],
                                    horizontal_spacing = 0,
                                    vertical_spacing=0)

colorbar_abs = dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = ['1', '10', '100', '1000'])
colorbar_pc = dict(len=1, x=1)

for var_i, var in enumerate(rules):
    if var_i == 0:
        cm= True
    else:
        cm= False
    fig.add_trace(
        go.Choropleth(
        locations=group_eu,
        z = data[var],
        locationmode = 'ISO-3',
        colorscale = 'RdBu',
        zmax = 12,
        zmin = -12,
        text = [str(r)+": {:0.2f} Gt CO<sub>2</sub>e".format(float(data.sel(Region=r)[var])) for r_i, r in enumerate(group_eu)],
        hovertemplate  = '%{text}',
        name="",
        marker_line_color='grey', 
        marker_line_width=0.7,
        colorbar=dict(len=1, x=1, tickvals = np.arange(-25, 25+0.1, 5), title={"text":"GHG budget<br>2021-2100<br>(Gt CO<sub>2/sub>e)"}, tickfont={'size': 25}),
        showscale=cm
    ), [1, 1, 1, 2, 2, 2][var_i], [1, 2, 3, 1, 2, 3][var_i])
# fig.update_geos(showocean=True, oceancolor="white")
# fig.update_geos(showlakes=True, lakecolor="white")

# limite map scope to USA 'africa', 'asia', 'europe', 'north america', 'south america', 'usa', 'world'
fig.update_layout(
    geo1 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo2 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo3 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo4 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo5 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo6 = dict(scope='europe', 
                showlakes=False,
                showcoastlines=False,),
)

mn = np.min(np.array([float(data[rules].min()[r]) for r in rules]))
mx = np.max(np.array([float(data[rules].max()[r]) for r in rules]))
for var_i, var in enumerate(rules):
    vals = np.array(data[var]).astype(float)
    vals_i = (vals + mn)**1.2
    vals_n = 14*(vals - mn)/(mx - mn)+7
    vals_boolean = np.copy(vals)
    vals_boolean[vals >= 0] = 1
    vals_boolean[vals < 0] = 0
    vals_boolean = vals_boolean.astype(int)
    
    fig.add_scattergeo(
                locations = group_eu,
                text = vals.round(1),
                featureidkey="properties.NAME_3",
                mode = 'text',
                hoverinfo='skip',
                showlegend=False,
                textfont={'size': 20,
                          'color': np.array(['black', 'black'])[vals_boolean]},
                row=[1, 1, 1, 2, 2, 2][var_i],
                col=[1, 2, 3, 1, 2, 3][var_i])

fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                resolution=50,
                lataxis_range=[30,75], lonaxis_range=[-20, 40],
                showcoastlines=False,)

#fig.update_traces(showscale=False)
fig.update_layout(height=1200, width=2300)
fig.update_geos(visible=True, showcountries=False)#fitbounds="locations",
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.07, text="<b>(a) Grandfathering</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.47, text="<b>(b) Per Capita</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.90, text="<b>(c) Per Capita Convergence</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.07, text="<b>(d) Equal Cumulative per Capita</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.48, text="<b>(e) Ability to Pay</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.95, text="<b>(f) Greenhouse Development Rights</b>")
fig_europe = fig
fig.show()

C:\Users\dekkerm\AppData\Local\Temp\3\ipykernel_6896\6629110.py:70: RuntimeWarning:

invalid value encountered in power



In [11]:
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/Europe_budget.png", scale=3)

In [12]:
data2 = data / xr_dataread.sel(Time=2021).GHG_hist*1e3

In [13]:
fig = make_subplots(rows=2, cols=3, #subplot_titles=('Grandfathering (GF)',
                                                    # 'Per Capita (PC)',
                                                    # 'Per Capita Convergence (PCC)',
                                                    # 'Equal Cumulative Per Capita',
                                                    # 'Ability to pay (AP)',
                                                    # 'Greenhouse Development Rights'), 
                                    specs=[[{'type': 'choropleth'}, {'type': 'choropleth'}, {'type': 'choropleth'}],
                                           [{'type': 'choropleth'}, {'type': 'choropleth'}, {'type': 'choropleth'}]],
                                    horizontal_spacing = 0,
                                    vertical_spacing=0)

colorbar_abs = dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = ['1', '10', '100', '1000'])
colorbar_pc = dict(len=1, x=1)

for var_i, var in enumerate(rules):
    if var_i == 0:
        cm= True
    else:
        cm= False
    fig.add_trace(
        go.Choropleth(
        locations=group_eu,
        z = data2[var],
        locationmode = 'ISO-3',
        colorscale = 'RdBu',
        zmax = 18,
        zmin = -18,
        text = [str(r)+": {:0.2f} Gt CO<sub>2</sub>e".format(float(data2.sel(Region=r)[var])) for r_i, r in enumerate(group_eu)],
        hovertemplate  = '%{text}',
        name="",
        marker_line_color='grey', 
        marker_line_width=0.7,
        colorbar=dict(len=1, x=1, tickvals = np.arange(-25, 25+0.1, 5), title={"text":"CO<sub>2</sub>e budget<br>2021-2100<br>expressed<br>in 2021<br>emissions"}, tickfont={'size': 25}),
        showscale=cm
    ), [1, 1, 1, 2, 2, 2][var_i], [1, 2, 3, 1, 2, 3][var_i])
# fig.update_geos(showocean=True, oceancolor="white")
# fig.update_geos(showlakes=True, lakecolor="white")

# limite map scope to USA 'africa', 'asia', 'europe', 'north america', 'south america', 'usa', 'world'
fig.update_layout(
    geo1 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo2 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo3 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo4 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo5 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo6 = dict(scope='europe', 
                showlakes=False,
                showcoastlines=False,),
)

mn = np.min(np.array([float(data2[rules].min()[r]) for r in rules]))
mx = np.max(np.array([float(data2[rules].max()[r]) for r in rules]))
for var_i, var in enumerate(rules):
    vals = np.array(data2[var]).astype(float)
    vals_i = (vals + mn)**1.2
    vals_n = 14*(vals - mn)/(mx - mn)+7
    vals_boolean = np.copy(vals)
    vals_boolean[vals >= 0] = 1
    vals_boolean[vals < 0] = 0
    vals_boolean = vals_boolean.astype(int)
    
    fig.add_scattergeo(
                locations = group_eu,
                text = vals.round(0),
                featureidkey="properties.NAME_3",
                mode = 'text',
                hoverinfo='skip',
                showlegend=False,
                textfont={'size': 20,
                          'color': np.array(['black', 'black'])[vals_boolean]},
                row=[1, 1, 1, 2, 2, 2][var_i],
                col=[1, 2, 3, 1, 2, 3][var_i])

fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                resolution=50,
                lataxis_range=[30,75], lonaxis_range=[-20, 40],
                showcoastlines=False,)

#fig.update_traces(showscale=False)
fig.update_layout(height=1200, width=2300)
fig.update_geos(visible=True, showcountries=False)#fitbounds="locations",
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.07, text="<b>(a) Grandfathering</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.47, text="<b>(b) Per Capita</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.90, text="<b>(c) Per Capita Convergence</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.07, text="<b>(d) Equal Cumulative per Capita</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.48, text="<b>(e) Ability to Pay</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.95, text="<b>(f) Greenhouse Development Rights</b>")
fig_europe = fig
fig.show()

C:\Users\dekkerm\AppData\Local\Temp\3\ipykernel_6896\2531417473.py:70: RuntimeWarning:

invalid value encountered in power



In [22]:
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/Europe_budget_rel.png", scale=3)

In [23]:
data3 = xrt.sel(**standard_settings, Time=2030, Region=group_eu).mean(dim='Scenario')

In [26]:
fig = make_subplots(rows=2, cols=3, #subplot_titles=('Grandfathering (GF)',
                                                    # 'Per Capita (PC)',
                                                    # 'Per Capita Convergence (PCC)',
                                                    # 'Equal Cumulative Per Capita',
                                                    # 'Ability to pay (AP)',
                                                    # 'Greenhouse Development Rights'), 
                                    specs=[[{'type': 'choropleth'}, {'type': 'choropleth'}, {'type': 'choropleth'}],
                                           [{'type': 'choropleth'}, {'type': 'choropleth'}, {'type': 'choropleth'}]],
                                    horizontal_spacing = 0,
                                    vertical_spacing=0)

colorbar_abs = dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = ['1', '10', '100', '1000'])
colorbar_pc = dict(len=1, x=1)

for var_i, var in enumerate(rules):
    if var_i == 0:
        cm= True
    else:
        cm= False
    fig.add_trace(
        go.Choropleth(
        locations=group_eu,
        z = data3[var],
        locationmode = 'ISO-3',
        colorscale = 'RdBu',
        zmax = 500,
        zmin = -500,
        text = [str(r)+": {:0.2f} Gt CO<sub>2</sub>e".format(float(data3.sel(Region=r)[var])) for r_i, r in enumerate(group_eu)],
        hovertemplate  = '%{text}',
        name="",
        marker_line_color='grey', 
        marker_line_width=0.7,
        colorbar=dict(len=1, x=1, tickvals = np.arange(-500, 500, 50), title={"text":"GHG<br>allocation<br>in 2030"}, tickfont={'size': 25}),
        showscale=cm
    ), [1, 1, 1, 2, 2, 2][var_i], [1, 2, 3, 1, 2, 3][var_i])
# fig.update_geos(showocean=True, oceancolor="white")
# fig.update_geos(showlakes=True, lakecolor="white")

# limite map scope to USA 'africa', 'asia', 'europe', 'north america', 'south america', 'usa', 'world'
fig.update_layout(
    geo1 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo2 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo3 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo4 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo5 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo6 = dict(scope='europe', 
                showlakes=False,
                showcoastlines=False,),
)

mn = np.min(np.array([float(data3[rules].min()[r]) for r in rules]))
mx = np.max(np.array([float(data3[rules].max()[r]) for r in rules]))
for var_i, var in enumerate(rules):
    vals = np.array(data3[var]).astype(float)
    vals_i = (vals + mn)**1.2
    vals_n = 14*(vals - mn)/(mx - mn)+7
    vals_boolean = np.copy(vals)
    vals_boolean[vals >= 0] = 1
    vals_boolean[vals < 0] = 0
    vals_boolean = vals_boolean.astype(int)
    
    fig.add_scattergeo(
                locations = group_eu,
                text = vals.round(0),
                featureidkey="properties.NAME_3",
                mode = 'text',
                hoverinfo='skip',
                showlegend=False,
                textfont={'size': 20,
                        'color': 'black'},
                row=[1, 1, 1, 2, 2, 2][var_i],
                col=[1, 2, 3, 1, 2, 3][var_i])

fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                resolution=50,
                lataxis_range=[30,75], lonaxis_range=[-20, 40],
                showcoastlines=False,)

#fig.update_traces(showscale=False)
fig.update_layout(height=1200, width=2300)
fig.update_geos(visible=True, showcountries=False)#fitbounds="locations",
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.07, text="<b>(a) Grandfathering</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.47, text="<b>(b) Per Capita</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.90, text="<b>(c) Per Capita Convergence</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.07, text="<b>(d) Equal Cumulative per Capita</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.48, text="<b>(e) Ability to Pay</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.95, text="<b>(f) Greenhouse Development Rights</b>")
fig_europe = fig
fig.show()

C:\Users\dekkerm\AppData\Local\Temp\3\ipykernel_6896\1687074859.py:70: RuntimeWarning:

invalid value encountered in power



In [27]:
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/Europe_2030.png", scale=3)

In [28]:
data4 = (xr_dataread.sel(Time=2021).GHG_hist - xrt.sel(**standard_settings, Time=2030, Region=group_eu).mean(dim='Scenario')) / xr_dataread.sel(Time=2021).GHG_hist

In [29]:
fig = make_subplots(rows=2, cols=3, #subplot_titles=('Grandfathering (GF)',
                                                    # 'Per Capita (PC)',
                                                    # 'Per Capita Convergence (PCC)',
                                                    # 'Equal Cumulative Per Capita',
                                                    # 'Ability to pay (AP)',
                                                    # 'Greenhouse Development Rights'), 
                                    specs=[[{'type': 'choropleth'}, {'type': 'choropleth'}, {'type': 'choropleth'}],
                                           [{'type': 'choropleth'}, {'type': 'choropleth'}, {'type': 'choropleth'}]],
                                    horizontal_spacing = 0,
                                    vertical_spacing=0)

colorbar_abs = dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = ['1', '10', '100', '1000'])
colorbar_pc = dict(len=1, x=1)

for var_i, var in enumerate(rules):
    if var_i == 0:
        cm= True
    else:
        cm= False
    fig.add_trace(
        go.Choropleth(
        locations=group_eu,
        z = data4[var],
        locationmode = 'ISO-3',
        colorscale = 'RdYlBu_r',
        zmax = 1,
        zmin = 0,
        text = [str(r)+": {:0.2f} Gt CO<sub>2</sub>e".format(float(data4.sel(Region=r)[var])) for r_i, r in enumerate(group_eu)],
        hovertemplate  = '%{text}',
        name="",
        marker_line_color='grey', 
        marker_line_width=0.7,
        colorbar=dict(len=1, x=1, tickvals = np.arange(0, 1, 0.1), ticktext = [str(int(i*100))+"%" for i in np.arange(0, 1, 0.1)], title={"text":"Emission<br>reduction<br>in 2030<br>w.r.t. 2021"}, tickfont={'size': 25}),
        showscale=cm
    ), [1, 1, 1, 2, 2, 2][var_i], [1, 2, 3, 1, 2, 3][var_i])
# fig.update_geos(showocean=True, oceancolor="white")
# fig.update_geos(showlakes=True, lakecolor="white")

# limite map scope to USA 'africa', 'asia', 'europe', 'north america', 'south america', 'usa', 'world'
fig.update_layout(
    geo1 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo2 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo3 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo4 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo5 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo6 = dict(scope='europe', 
                showlakes=False,
                showcoastlines=False,),
)

mn = np.min(np.array([float(data4[rules].min()[r]) for r in rules]))
mx = np.max(np.array([float(data4[rules].max()[r]) for r in rules]))
for var_i, var in enumerate(rules):
    vals = np.array(data4[var]).astype(float)
    vals_i = (vals + mn)**1.2
    vals_n = 14*(vals - mn)/(mx - mn)+7
    vals_boolean = np.copy(vals)
    vals_boolean[vals >= 0] = 1
    vals_boolean[vals < 0] = 0
    vals_boolean = vals_boolean.astype(int)
    vals[np.isnan(vals)] = np.nanmean(vals)
    
    fig.add_scattergeo(
                locations = group_eu,
                text = [str(np.int(vals[r_i]*100))+"%" for r_i in range(len(group_eu))],
                featureidkey="properties.NAME_3",
                mode = 'text',
                hoverinfo='skip',
                showlegend=False,
                textfont={'size': 20,
                        'color': 'black'},
                row=[1, 1, 1, 2, 2, 2][var_i],
                col=[1, 2, 3, 1, 2, 3][var_i])

fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                resolution=50,
                lataxis_range=[30,75], lonaxis_range=[-20, 40],
                showcoastlines=False,)

#fig.update_traces(showscale=False)
fig.update_layout(height=1200, width=2300)
fig.update_geos(visible=True, showcountries=False)#fitbounds="locations",
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.07, text="<b>(a) Grandfathering</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.47, text="<b>(b) Per Capita</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.90, text="<b>(c) Per Capita Convergence</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.07, text="<b>(d) Equal Cumulative per Capita</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.48, text="<b>(e) Ability to Pay</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.95, text="<b>(f) Greenhouse Development Rights</b>")
fig_europe = fig
fig.show()

C:\Users\dekkerm\AppData\Local\Temp\3\ipykernel_6896\853437972.py:80: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\dekkerm\AppData\Local\Temp\3\ipykernel_6896\853437972.py:70: RuntimeWarning:

invalid value encountered in power



In [30]:
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/Europe_2030perc.png", scale=3)

In [31]:
data5 = (xr_dataread.sel(Time=2021).GHG_hist - xrt.sel(**standard_settings, Time=2040, Region=group_eu).mean(dim='Scenario')) / xr_dataread.sel(Time=2021).GHG_hist

In [32]:
fig = make_subplots(rows=2, cols=3, #subplot_titles=('Grandfathering (GF)',
                                                    # 'Per Capita (PC)',
                                                    # 'Per Capita Convergence (PCC)',
                                                    # 'Equal Cumulative Per Capita',
                                                    # 'Ability to pay (AP)',
                                                    # 'Greenhouse Development Rights'), 
                                    specs=[[{'type': 'choropleth'}, {'type': 'choropleth'}, {'type': 'choropleth'}],
                                           [{'type': 'choropleth'}, {'type': 'choropleth'}, {'type': 'choropleth'}]],
                                    horizontal_spacing = 0,
                                    vertical_spacing=0)

colorbar_abs = dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = ['1', '10', '100', '1000'])
colorbar_pc = dict(len=1, x=1)

for var_i, var in enumerate(rules):
    if var_i == 0:
        cm= True
    else:
        cm= False
    fig.add_trace(
        go.Choropleth(
        locations=group_eu,
        z = data5[var],
        locationmode = 'ISO-3',
        colorscale = 'RdYlBu_r',
        zmax = 1,
        zmin = 0,
        text = [str(r)+": {:0.2f} Gt CO<sub>2</sub>e".format(float(data5.sel(Region=r)[var])) for r_i, r in enumerate(group_eu)],
        hovertemplate  = '%{text}',
        name="",
        marker_line_color='grey', 
        marker_line_width=0.7,
        colorbar=dict(len=1, x=1, tickvals = np.arange(0, 1, 0.1), ticktext = [str(int(i*100))+"%" for i in np.arange(0, 1, 0.1)], title={"text":"Emission<br>reduction<br>in 2040<br>w.r.t. 2021"}, tickfont={'size': 25}),
        showscale=cm
    ), [1, 1, 1, 2, 2, 2][var_i], [1, 2, 3, 1, 2, 3][var_i])
# fig.update_geos(showocean=True, oceancolor="white")
# fig.update_geos(showlakes=True, lakecolor="white")

# limite map scope to USA 'africa', 'asia', 'europe', 'north america', 'south america', 'usa', 'world'
fig.update_layout(
    geo1 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo2 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo3 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo4 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo5 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo6 = dict(scope='europe', 
                showlakes=False,
                showcoastlines=False,),
)

mn = np.min(np.array([float(data5[rules].min()[r]) for r in rules]))
mx = np.max(np.array([float(data5[rules].max()[r]) for r in rules]))
for var_i, var in enumerate(rules):
    vals = np.array(data5[var]).astype(float)
    vals_i = (vals + mn)**1.2
    vals_n = 14*(vals - mn)/(mx - mn)+7
    vals_boolean = np.copy(vals)
    vals_boolean[vals >= 0] = 1
    vals_boolean[vals < 0] = 0
    vals_boolean = vals_boolean.astype(int)
    vals[np.isnan(vals)] = np.nanmean(vals)
    
    fig.add_scattergeo(
                locations = group_eu,
                text = [str(np.int(vals[r_i]*100))+"%" for r_i in range(len(group_eu))],
                featureidkey="properties.NAME_3",
                mode = 'text',
                hoverinfo='skip',
                showlegend=False,
                textfont={'size': 20,
                        'color': 'black'},
                row=[1, 1, 1, 2, 2, 2][var_i],
                col=[1, 2, 3, 1, 2, 3][var_i])

fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                resolution=50,
                lataxis_range=[30,75], lonaxis_range=[-20, 40],
                showcoastlines=False,)

#fig.update_traces(showscale=False)
fig.update_layout(height=1200, width=2300)
fig.update_geos(visible=True, showcountries=False)#fitbounds="locations",
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.07, text="<b>(a) Grandfathering</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.47, text="<b>(b) Per Capita</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.90, text="<b>(c) Per Capita Convergence</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.07, text="<b>(d) Equal Cumulative per Capita</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.48, text="<b>(e) Ability to Pay</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.95, text="<b>(f) Greenhouse Development Rights</b>")
fig_europe = fig
fig.show()

C:\Users\dekkerm\AppData\Local\Temp\3\ipykernel_6896\2992891589.py:80: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [33]:
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/Europe_2040perc.png", scale=3)

In [34]:
data5= xr_temps.sel(**standard_settings, Risk=0.5, Region=group_eu).mean(dim=['Conditionality', 'Hot_air', 'Ambition', 'Scenario'])

In [35]:
fig = make_subplots(rows=2, cols=3, #subplot_titles=('Grandfathering (GF)',
                                                    # 'Per Capita (PC)',
                                                    # 'Per Capita Convergence (PCC)',
                                                    # 'Equal Cumulative Per Capita',
                                                    # 'Ability to pay (AP)',
                                                    # 'Greenhouse Development Rights'), 
                                    specs=[[{'type': 'choropleth'}, {'type': 'choropleth'}, {'type': 'choropleth'}],
                                           [{'type': 'choropleth'}, {'type': 'choropleth'}, {'type': 'choropleth'}]],
                                    horizontal_spacing = 0,
                                    vertical_spacing=0)

colorbar_abs = dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = ['1', '10', '100', '1000'])
colorbar_pc = dict(len=1, x=1)

for var_i, var in enumerate(rules):
    if var_i == 0:
        cm= True
    else:
        cm= False
    fig.add_trace(
        go.Choropleth(
        locations=group_eu,
        z = data5[var],
        locationmode = 'ISO-3',
        colorscale = 'RdYlGn_r',
        zmax = 2.5,
        zmin = 1.5,
        text = [str(r)+": {:0.2f} deg C".format(float(data5.sel(Region=r)[var])) for r_i, r in enumerate(group_eu)],
        hovertemplate  = '%{text}',
        name="",
        marker_line_color='grey', 
        marker_line_width=0.7,
        colorbar=dict(len=1, x=1, tickvals = np.arange(1.5, 2.5, 0.1), ticktext = [str(np.round(i,1))+"&deg;C" for i in np.arange(1.5, 2.5, 0.1)], title={"text":"Temp-equiv<br>of NDC"}, tickfont={'size': 25}),
        showscale=cm
    ), [1, 1, 1, 2, 2, 2][var_i], [1, 2, 3, 1, 2, 3][var_i])
# fig.update_geos(showocean=True, oceancolor="white")
# fig.update_geos(showlakes=True, lakecolor="white")

# limite map scope to USA 'africa', 'asia', 'europe', 'north america', 'south america', 'usa', 'world'
fig.update_layout(
    geo1 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo2 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo3 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo4 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo5 = dict(scope='europe', 
                showlakes=False,
                lakecolor='rgb(255, 255, 255)',
                showcoastlines=False,),
    geo6 = dict(scope='europe', 
                showlakes=False,
                showcoastlines=False,),
)

mn = np.min(np.array([float(data5[rules].min()[r]) for r in rules]))
mx = np.max(np.array([float(data5[rules].max()[r]) for r in rules]))
for var_i, var in enumerate(rules):
    vals = np.array(data5[var]).astype(float)
    vals_i = (vals + mn)**1.2
    vals_n = 14*(vals - mn)/(mx - mn)+7
    vals_boolean = np.copy(vals)
    vals_boolean[vals >= 0] = 1
    vals_boolean[vals < 0] = 0
    vals_boolean = vals_boolean.astype(int)
    
    fig.add_scattergeo(
                locations = group_eu,
                text = [str(np.round(vals[r_i], 1)) for r_i in range(len(group_eu))],
                featureidkey="properties.NAME_3",
                mode = 'text',
                hoverinfo='skip',
                showlegend=False,
                textfont={'size': 20,
                        'color': 'black'},
                row=[1, 1, 1, 2, 2, 2][var_i],
                col=[1, 2, 3, 1, 2, 3][var_i])

fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                resolution=50,
                lataxis_range=[30,75], lonaxis_range=[-20, 40],
                showcoastlines=False,)

#fig.update_traces(showscale=False)
fig.update_layout(height=1200, width=2300)
fig.update_geos(visible=True, showcountries=False)#fitbounds="locations",
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.07, text="<b>(a) Grandfathering</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.47, text="<b>(b) Per Capita</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.96, x=0.90, text="<b>(c) Per Capita Convergence</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.07, text="<b>(d) Equal Cumulative per Capita</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.48, text="<b>(e) Ability to Pay</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=25),
                   y=0.45, x=0.95, text="<b>(f) Greenhouse Development Rights</b>")
fig_europe = fig
fig.show()

In [36]:
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/Europe_temp.png", scale=3)